## Environment setup

In [ ]:
!pip install chromadb langchain openai pypdf tqdm

## Imports

In [44]:
from pypdf import PdfReader
import re

## Variables

In [45]:
criminal_code = 'criminal_code_of_ukraine.pdf'

## Parse PDF into raw text

In [46]:
def load_pdf_text(path: str) -> str:
    reader = PdfReader(path)

    pages = []

    for page in reader.pages:
        text = page.extract_text()

        if text:
            pages.append(text)

    return '\n'.join(pages)


criminal_code_raw_text = load_pdf_text(criminal_code)

In [47]:
print(criminal_code_raw_text)

КРИМІНАЛЬНИЙ КОДЕКС УКРАЇНИ
(Відомості Верховної Ради України (ВВР), 2001, № 25-26, ст.131)
{Із змінами, внесеними згідно із Законами 
№ 2953-III від 17.01.2002, ВВР, 2002, № 17, ст.121 
№ 3075-III від 07.03.2002, ВВР, 2002, № 30, ст.206 
№ 430-IV від 16.01.2003, ВВР, 2003, № 14, ст.95 
№ 485-IV від 06.02.2003, ВВР, 2003, № 14, ст.104 
№ 662-IV від 03.04.2003, ВВР, 2003, № 27, ст.209 
№ 668-IV від 03.04.2003, ВВР, 2003, № 26, ст.198 
№ 669-IV від 03.04.2003, ВВР, 2003, № 26, ст.199 
№ 744-IV від 15.05.2003, ВВР, 2003, № 29, ст.234 
№ 850-IV від 22.05.2003, ВВР, 2003, № 35, ст.271 
№ 908-IV від 05.06.2003, ВВР, 2003, № 38, ст.320 
№ 1098-IV від 10.07.2003, ВВР, 2004, № 7, ст.46 
№ 1130-IV від 11.07.2003, ВВР, 2004, № 8, ст.66 
№ 1626-IV від 18.03.2004, ВВР, 2004, № 26, ст.361 
№ 1723-IV від 18.05.2004, ВВР, 2004, № 36, ст.430}
{Щодо визнання неконституційними окремих положень див. Рішення 
Конституційного Суду 
№ 15-рп/2004 від 02.11.2004}
{Із змінами, внесеними згідно із Законами 
№ 22

## remove header

In [48]:
def remove_header(text: str) -> str:
    try:
        marker = "ЗАГАЛЬНА ЧАСТИНА"
        idx = text.find(marker)
        print('idx:', idx)

        if idx == -1:
            return text

        return text[idx:]
    except Exception as e:
        print(f"error while finding header marker: {e}")
        return text


raw_text_without_header = remove_header(criminal_code_raw_text)
print(raw_text_without_header)

idx: 15377
ЗАГАЛЬНА ЧАСТИНА
Розділ I 
ЗАГАЛЬНІ ПОЛОЖЕННЯ
Стаття 1. Завдання Кримінального кодексу України
1. Кримінальний кодекс України має своїм завданням правове забезпечення охорони 
прав і свобод людини і громадянина, власності, громадського порядку та громадської 
безпеки, довкілля, конституційного устрою України від кримінально-протиправних 
посягань, забезпечення миру і безпеки людства, а також запобігання кримінальним 
правопорушенням.
2. Для здійснення цього завдання Кримінальний кодекс України визначає, які суспільно 
небезпечні діяння є кримінальними правопорушеннями та які покарання застосовуються до 
осіб, що їх вчинили.
{Стаття 1 із змінами, внесеними згідно із Законом № 2617-VIII від 22.11.2018}
Стаття 2. Підстава кримінальної відповідальності
1. Підставою кримінальної відповідальності є вчинення особою суспільно 
небезпечного діяння, яке містить склад кримінального правопорушення, передбаченого 
цим Кодексом.
2. Особа вважається невинуватою у вчиненні кримінального пра

## split main and footer

In [ ]:
# 1. separate main part and footer
# 2. remove all info before ЗАГАЛЬНА ЧАСТИНА
# 3. store footer info:  Кримінальний кодекс України Кодекс України; Кодекс, Закон від 05.04.2001 № 2341-III Редакція від 17.07.2025, підстава — 4496-IX, 4499-IX Постійна адреса: https://zakon.rada.gov.ua/go/2341-14 Законодавство України станом на 09.02.2026 чинний

#   fields to store:
# - official_source: https://zakon.rada.gov.ua/go/2341-14
# - adoption_date: 2001-04-05
# - edition_date: 2025-07-17
# - document_number: 2341-III

In [49]:
def split_main_and_footer(text: str):
    """
    splits text into: main_text: everything BEFORE 'Президент України Л.КУЧМА ...' footer_text: everything FROM 'Президент України Л.КУЧМА ...' to the end
    """

    marker = 'Президент України Л.КУЧМА'
    idx = text.find(marker)

    if idx == -1:
        raise Exception(f"Marker '{marker}' not found in text")

    main_text = text[:idx]
    footer_text = text[idx:]

    return main_text, footer_text


main_txt, footer_txt = split_main_and_footer(raw_text_without_header)

print("main_txt (beginning): \n", main_txt[:500])
print("main_txt (end): \n", main_txt[-500:])

print("footer_txt (beginning):  \n", footer_txt[:500])
print("footer_txt (end): \n", footer_txt[-500:])

main_txt (beginning): 
 ЗАГАЛЬНА ЧАСТИНА
Розділ I 
ЗАГАЛЬНІ ПОЛОЖЕННЯ
Стаття 1. Завдання Кримінального кодексу України
1. Кримінальний кодекс України має своїм завданням правове забезпечення охорони 
прав і свобод людини і громадянина, власності, громадського порядку та громадської 
безпеки, довкілля, конституційного устрою України від кримінально-протиправних 
посягань, забезпечення миру і безпеки людства, а також запобігання кримінальним 
правопорушенням.
2. Для здійснення цього завдання Кримінальний кодекс України в
main_txt (end): 
 дно із Законом № 2114-IX від 03.03.2022}
23. З дня набрання чинності Законом України "Про внесення змін до Кримінального, 
Кримінального процесуального кодексів України та інших законодавчих актів України 
щодо удосконалення видів кримінальних покарань" засудженим, які відбувають покарання 
у виді арешту в арештних домах, замінити невідбуту частину покарання на інший вид 
покарання відповідно до статті 72 цього Кодексу.
{Розділ II доповнено пунктом 23 з

## split main into 2 parts: Загальна частина, Особлива частина and Перехідні та прикінцеві положення
Загальна частина and Особлива частина have the same structure.

In [50]:
def split_main_into_2_parts(main_text: str):
    marker = 'ПРИКІНЦЕВІ ТА ПЕРЕХІДНІ ПОЛОЖЕННЯ'

    idx = main_txt.find(marker)

    if idx == -1:
        raise Exception(f"Marker '{marker}' not found in text")

    first_part_of_text = main_text[:idx]
    second_part_of_text = main_text[idx:]

    return first_part_of_text, second_part_of_text


first_part_of_txt, second_part_of_txt = split_main_into_2_parts(main_txt)

print("first part of txt (beginning): \n", first_part_of_txt[:500])
print("first part of txt (end): \n", first_part_of_txt[-500:])

print("second part of txt (beginning):  \n", second_part_of_txt[:500])
print("second part of txt (end): \n", second_part_of_txt[-500:])

first part of txt (beginning): 
 ЗАГАЛЬНА ЧАСТИНА
Розділ I 
ЗАГАЛЬНІ ПОЛОЖЕННЯ
Стаття 1. Завдання Кримінального кодексу України
1. Кримінальний кодекс України має своїм завданням правове забезпечення охорони 
прав і свобод людини і громадянина, власності, громадського порядку та громадської 
безпеки, довкілля, конституційного устрою України від кримінально-протиправних 
посягань, забезпечення миру і безпеки людства, а також запобігання кримінальним 
правопорушенням.
2. Для здійснення цього завдання Кримінальний кодекс України в
first part of txt (end): 
 е є ні громадянином (підданим) сторони, що перебуває у конфлікті, ні особою, яка 
постійно на законних підставах проживає на території, яка контролюється стороною, що 
перебуває у конфлікті;
4) не входить до особового складу збройних сил держави, на території якої 
здійснюються такі дії;
5) не послана державою, яка не є стороною, що перебуває у конфлікті, для виконання 
офіційних обов’язків як особи, яка входить до складу її збройних с

## Normalization

### normalize the main text

In [51]:
def normalize_apostrophes(text: str) -> str:
    text = text.replace("'", "’")
    text = text.replace("ʼ", "’")

    return text

### specify regular expressions

In [75]:
def fix_line_breaks(text: str) -> str:
    # Fix hyphenated words split over lines
    text = re.sub(r"(\w+)-\s*\n\s*(\w+)", r"\1-\2", text)

    # Join ALL-CAPS titles and regular lines broken mid-sentence
    text = re.sub(r"([^\n])\n([^\n])", r"\1 \2", text)

    # Clean up spaces
    text = re.sub(r" +\n", "\n", text)
    text = re.sub(r"\n +", "\n", text)
    text = re.sub(r"  +", " ", text)

    return text


def enforce_headings(text: str) -> str:
    # Major parts
    text = re.sub(r"\s*(ЗАГАЛЬНА ЧАСТИНА|ОСОБЛИВА ЧАСТИНА|ПЕРЕХІДНІ ТА ПРИКІНЦЕВІ ПОЛОЖЕННЯ)\s*", r"\n\n\1\n\n", text)

    # Sections (only if NOT preceded by "{")
    text = re.sub(r"(?<!\{)\s*(Розділ [IVXLC]+(?:\s*-\s*\d+)?)\s*\.?\s*", r"\n\1\n", text)

    # Section titles (ALL CAPS) - ensure they're on their own line
    text = re.sub(r"([А-ЯІЇЄҐ][А-ЯІЇЄҐ\s,]{3,})\s+(Стаття|\{)", r"\1\n\n\2", text)

    # Articles - newline before Стаття that has a title (not just deletion notes)
    text = re.sub(r"(\})\s*(Стаття \d+[-\d]*\.\s+[А-ЯІЇЄҐА-я])", r"\1\n\2", text)
    text = re.sub(r"\s*(Стаття \d+[-\d]*\.)\s+([А-ЯІЇЄҐ])", r"\n\1 \2", text)

    # Cleanup
    text = re.sub(r"\n{3,}", "\n\n", text)

    return text.strip()

### normalize first part of main text: Загальна частина, Особлива частина

In [76]:
normalized_text = fix_line_breaks(first_part_of_txt)
result = enforce_headings(normalized_text)
print(result)

ЗАГАЛЬНА ЧАСТИНА
Розділ I
ЗАГАЛЬНІ ПОЛОЖЕННЯ
Стаття 1. Завдання Кримінального кодексу України 1. Кримінальний кодекс України має своїм завданням правове забезпечення охорони прав і свобод людини і громадянина, власності, громадського порядку та громадської безпеки, довкілля, конституційного устрою України від кримінально-протиправних посягань, забезпечення миру і безпеки людства, а також запобігання кримінальним правопорушенням. 2. Для здійснення цього завдання Кримінальний кодекс України визначає, які суспільно небезпечні діяння є кримінальними правопорушеннями та які покарання застосовуються до осіб, що їх вчинили. {Стаття 1 із змінами, внесеними згідно із Законом № 2617-VIII від 22.11.2018}
Стаття 2. Підстава кримінальної відповідальності 1. Підставою кримінальної відповідальності є вчинення особою суспільно небезпечного діяння, яке містить склад кримінального правопорушення, передбаченого цим Кодексом. 2. Особа вважається невинуватою у вчиненні кримінального правопорушення і не мож

### normalize 2-nd part of main text: Перехідні та прикінцеві положення

In [74]:
normalized_text = fix_line_breaks(second_part_of_txt)
result = enforce_headings(normalized_text)
print(result)

ПРИКІНЦЕВІ ТА ПЕРЕХІДНІ ПОЛОЖЕННЯ
Розділ I
1. Цей Кодекс набирає чинності з 1 вересня 2001 року. 2. З набранням чинності цим Кодексом втрачають чинність: Кримінальний кодекс Української РСР від 28 грудня 1960 року (Відомості Верховної Ради УРСР, 1961 р., № 2, ст. 14) із змінами, внесеними до нього, крім Переліку майна, що не підлягає конфіскації за судовим вироком (Додаток до цього Кодексу); Закон Української РСР "Про затвердження Кримінального кодексу Української РСР" (Відомості Верховної Ради УРСР, 1961 р., № 2, ст. 14); статті 1, 2 та 5 Указу Президії Верховної Ради Української РСР від 20 квітня 1990 року "Про відповідальність за дії, спрямовані проти громадського порядку і безпеки громадян" (Відомості Верховної Ради УРСР, 1990 р., № 18, ст. 278); Указ Президії Верховної Ради Української РСР від 26 грудня 1990 року "Про відповідальність за порушення порядку користування картками споживача на право придбання товарів та іншими офіційними документами" (Відомості Верховної Ради УРСР, 19